# Plotting X-CRR with loss and variable ring lengths

The following notebook will plot the reflectivity $R = |A_1^-|^2$ of the X-CRR numerically based on the following equation system

$$
\large
\begin{bmatrix}
    B^{\mp}_1 \\
    B^{\mp}_2 \\
    \end{bmatrix}
    =
    \frac{1}{-\kappa_A\kappa_B\tau_C}
    \begin{bmatrix}
    \tau_B & -1 \\
    T_B & -\tau_B
    \end{bmatrix}
    \begin{bmatrix}
    e^{\mp (j \beta - \alpha) (L_2^A+L_1^B)} & \mp j \kappa_C e^{\pm (j \beta - \alpha) (L_1^A - L_2^A)}
    \\
    \pm j \kappa_C e^{\mp (j \beta - \alpha) (L_1^B-L_2^B)} & T_C e^{\pm (j \beta - \alpha) (L_1^A+L_2^B)}
    \end{bmatrix}
    \begin{bmatrix}
    -\tau_A & 1 \\
    -T_A & \tau_A
    \end{bmatrix}
    \begin{bmatrix}
    A^{\pm}_1 \\
    A^{\pm}_2 \\
\end{bmatrix}
$$

$$
B_1^{\pm} = A_2^{\pm}\exp{ \left(\pm (j\beta_w-\alpha) L_w\right) } 
$$

With the excitation conditions $A_1^+=1$ and $B_2^-=0$



## Libraries

In [1]:
import numpy as np
from scipy.constants import c #speed of light
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator,FuncFormatter)
%matplotlib ipympl

## Functions

In [2]:
def xcrr_matrix(
    k_r,k_w,a,
    kappa_a, kappa_b, kappa_c,
    L1A, L1B, L2A, L2B,L_w
    ):
    """
    xcrr_matrix()
    Calculate the associated matrix for the X-CRR Device
    
    Inputs (in order):
    k_r: ring wavenumber = 2*pi*n_eff_r/wl
    k_w: waveguide wavenumber = 2*pi*n_eff_r/wl
    a: loss factor (um^-1)
    kappa_a/b/c: cross-coupling coefficient
    L1A: inner length of ring 1
    L1B: outer length of ring 1
    L2A: inner length of ring 2
    L2B: outer length of ring 2
    L_w: bus waveguide length
    
    Outputs:
    4x4 coefficient matrix
    """

    # Calculate full lengths L1 and L2
    L1 = L1A + L1B
    L2 = L2A + L2B

    # Calculate corresponding transmission coupling coefficient
    tau_a = np.sqrt(1-kappa_a**2)
    tau_b = np.sqrt(1-kappa_b**2)
    tau_c = np.sqrt(1-kappa_c**2)

    # Define complex kappa terms 
    jkappa_a = complex(0,kappa_a)
    jkappa_b = complex(0,kappa_b)
    jkappa_c = complex(0,kappa_c)


    # Define a short function that returns the propagation exponential terms given k, alpha, and propagation length. 
    # _p and _n refer to positive and negative signs
    def exp_r_p(L):
        return np.exp(complex(-a,k_r)*L)
    def exp_r_n(L):
        return np.exp(-1*complex(-a,k_r)*L)



    # Define the 3 matrices (1,2,3 from left to right)
    matrix1 = np.array(
        [
            np.array([tau_b,-1      ,0    ,0       ],dtype="complex"),
            np.array([1    ,-1*tau_b,0    ,0       ],dtype="complex"),
            np.array([0    ,0       ,tau_b,-1      ],dtype="complex"),
            np.array([0    ,0       ,1    ,-1*tau_b],dtype="complex"),
        ],
        dtype = "complex_"
    )

    matrix2 = np.array(
        [
            np.array([exp_r_p(L2A+L1B)          ,jkappa_c*(exp_r_n(L1A-L2A)),0                        ,0                           ],dtype="complex"),
            np.array([-jkappa_c*exp_r_p(L1B-L2B),exp_r_n(L1A+L2B)           ,0                        ,0                           ],dtype="complex"),
            np.array([0                         ,0                          ,exp_r_n(L2A+L1B)         ,-jkappa_c*(exp_r_p(L1A-L2A))],dtype="complex"),
            np.array([0                         ,0                          ,jkappa_c*exp_r_n(L1B-L2B),exp_r_p(L1A+L2B)            ],dtype="complex"),
        ],
        dtype = "complex_"
    )


    matrix3 = np.array(
        [
            np.array([-tau_a,1    ,0     ,0    ],dtype="complex"),
            np.array([-1    ,tau_a,0     ,0    ],dtype="complex"),
            np.array([0     ,0    ,-tau_a,1    ],dtype="complex"),
            np.array([0     ,0    ,-1    ,tau_a],dtype="complex"),
        ],
        dtype = "complex_"
    )
    
    matrix_const = 1/(-1*kappa_a*kappa_b*tau_c)

    return_matrix = matrix_const* (matrix1 @ matrix2 @ matrix3)

    return return_matrix

In [3]:
def calculate_coeffmatrix(
        x,
        n_r, n_w, a,
        kappa_a, kappa_b, kappa_c,
        L1A, L1B, L2A, L2B,L_w, 
        x_option = "wl"
    ):
    """
    calculate_coeffmatrix
    Calculate coefficient matrix for a given variable x
    
    Inputs (in order):
    x: variable (meaning is specified by x_option)
    n_r: ring effective index
    n_w: bus waveguide effective index,
    a: loss factor 
    kappa_a/b/c: cross-coupling coefficient
    L1A: inner length of ring 1
    L1B: outer length of ring 1
    L2A: inner length of ring 2
    L2B: outer length of ring 2
    L_w: bus waveguide length
    x_option: wl for wavelength [m], k for vacuum wavenumber, f for frequency [hz]
    
    Output: 
    4x4 coefficient matrix
    
    """

    # Define x-options as "k" (wavenumber), "wl" (wavelength), f (frequency) or norm_f (normalized frequency)
    x_options = {
        "k":"wavenumber (vacuum)",
        "wl": "wavelength [m]",
        "f":"frequency [Hz]"
    }
    

    # Define variables for ring and waveguide wave propagation constants
    if x_option == "k":
        k_r = n_r*x
        k_w = n_w*x
    elif x_option == "wl":
        omega = (2*np.pi*c)/x #Calculate angular frequency assuming x is wavelength
        k = omega/c #Calculate wavenumber
        k_r = n_r*k #Replace exisiting k_r and k_w values
        k_w = n_w*k
    elif x_option == "f":
        omega = (2*np.pi*x) #Calculate angular frequency assuming x is wavelength
        k = omega/c #Calculate wavenumber
        k_r = n_r*k #Replace exisiting k_r and k_w values
        k_w = n_w*k 
    else:
        raise Exception(r'k_r and k_w were not calcualted')


    return_matrix = xcrr_matrix(k_r,k_w, a, kappa_a, kappa_b, kappa_c, L1A, L1B, L2A, L2B, L_w)
    
    return return_matrix

In [4]:
def calculate_field(
        x,
        n_r, n_w,a,
        kappa_a, kappa_b, kappa_c,
        L1A, L1B, L2A, L2B,L_w,
        field_option = "R", x_option = "k", power = True
    ):
    """
    calculate reflected (transmitted) complex field or power for the 
    specificied x and device paramets
        
    Inputs (in order):
    x: variable (meaning is specified by x_option)
    n_r: ring effective index
    n_w: bus waveguide effective index,
    a: loss factor 
    kappa_a/b/c: cross-coupling coefficient
    L1A: inner length of ring 1
    L1B: outer length of ring 1
    L2A: inner length of ring 2
    L2B: outer length of ring 2
    L_w: bus waveguide length
    x_option: wl for wavelength [m], k for vacuum wavenumber, f for frequency [hz]
    field_option: R for reflection, T for transmission
    power: bool, True to return power, False to return complex field
    
    """
    
    #Define field options
    field_options = {
        "R":"Reflection",
        "T": "Transmission"
    }
    

    # Define x-options as "k" (wavenumber), "wl" (wavelength), f (frequency) or norm_f (normalized frequency)
    x_options = {
        "k":"wavenumber (vacuum)",
        "wl": "wavelength [m]",
        "f":"frequency [Hz]",
        "norm_f": "normalized frequency [rads]"
    }
    
    # Throw error when field_option is invalid
    if field_option not in field_options:
        raise Exception('field_option is invalid. Valid options are "R" (reflection) or "T" (transmission)')
    
    # Throw error when x_option is invalid
    if x_option not in x_options.keys():
        raise Exception(r'x_option is not valid. Valid options are "k" (wavenumber), "wl" (wavelength [m]), "f" (frequency [Hz]) or "norm_f" (normalized frequency [rads])')
    
    # Define variables for ring and waveguide wave propagation constants
    if x_option == "k":
        k_r = n_r*x
        k_w = n_w*x
    elif x_option == "wl":
        omega = (2*np.pi*c)/x #Calculate angular frequency assuming x is wavelength
        k = omega/c #Calculate wavenumber
        k_r = n_r*k #Replace exisiting k_r and k_w values
        k_w = n_w*k
    elif x_option == "f":
        omega = (2*np.pi*x) #Calculate angular frequency assuming x is wavelength
        k = omega/c #Calculate wavenumber
        k_r = n_r*k #Replace exisiting k_r and k_w values
        k_w = n_w*k 
    else:
        k_r = k_w = None
    
    
    # Define the propagation exponential term for the bus waveguide. p refers to the positive sign.
    exp_w_p = np.exp(complex(-a,k_w)*L_w)

    # Calculate matrix M depending on device_option passed
    M = calculate_coeffmatrix(x, n_r, n_w,a, kappa_a, kappa_b, kappa_c, L1A, L1B, L2A, L2B, L_w, x_option)
    
    # Calculate field numerator depending on field_option
    if field_option == "R":
        numerator = M[0,1]*M[2,3]*M[3,2] - M[0,1]*M[2,2]*M[3,3] - M[3,2]
    elif field_option == "T":
        numerator = M[1,0]*( M[0,1]*M[2,3]*M[3,2] - M[0,1]*M[2,2]*M[3,3] -  M[3,2] ) + M[0,0]*M[1,1]*( M[2,2]*M[3,3] - M[2,3]*M[3,2])
    else:
        raise Exception("Field numerator was not calculated")
        
    denominator = M[0,0]*M[3,3]
    
    field = (numerator/denominator)*(exp_w_p)
    power = np.power(np.abs(field),2)
    
    if power:
        return power
    else:
        return field

## Interactive Plot

In [5]:
# Define initial parameters that will stay constant. Change this manually,

# Paramters of the first ring
L1 = (100 * 2 * np.pi) * 1e-6 #microns
L1A = L1/4 
L1B = L1 - L1A

# Keep L2A the same but add a delta to L2B (as a slider)
L2A = L1A

# Length of bus waveguide 
L_w = 50*2*1e-6

# Refractive index
n_r = n_w = 3.29

# range of plot
wl_start = 1.5860e-6
wl_end = 1.58950e-6
wl_resolution = 1000

# wavelength array
wl_x = np.linspace(wl_start,wl_end,wl_resolution)


#Define initial parameters that will be varied later on

init_kappa_a = init_kappa_b = 0.5
init_kappa_c = 0.5
init_a = 0

# add a delta to L2B
init_deltaL = 0 # Expressed as a fraction of L1B
init_L2B = L1B + init_deltaL*L1B


fig, ax = plt.subplots()

init_power_array_lossless = np.empty(wl_x.shape)
for i, wl in enumerate(wl_x):

    # The other two variables are the complex field and the phase
    power_lossless = calculate_field(wl, 
                       n_r, n_w, 0,
                       init_kappa_a, init_kappa_b, init_kappa_c, 
                       L1A, L1B, L2A, init_L2B, L_w, 
                       field_option='R', 
                       x_option='wl', 
                      )
    init_power_array_lossless[i] = power_lossless
R_lossless_line, = ax.plot(wl_x*1e6, init_power_array_lossless, lw=1, c="tab:grey",alpha=0.5)


init_power_array = np.empty(wl_x.shape)
for i, wl in enumerate(wl_x):

    # The other two variables are the complex field and the phase
    power = calculate_field(wl, 
                       n_r, n_w, init_a,
                       init_kappa_a, init_kappa_b, init_kappa_c, 
                       L1A, L1B, L2A, init_L2B, L_w, 
                       field_option='R', 
                       x_option='wl', 
                      )
    init_power_array[i] = power
R_line, = ax.plot(wl_x*1e6, init_power_array, lw=1, c="tab:orange")

# Fix axes
ax.set_xlabel(r"Wavelength [$\mu$m]")
ax.set_ylabel("Reflectivity $R=|A_1^-|^2$")
ax.grid(True, linestyle = "dashed")
ax.grid(True, which = "minor",linestyle = "dashed")

# Adjust the main plot to make room for the sliders
plt.subplots_adjust(left=0.2, bottom=0.45,top=0.95)

# Define colors and margins 
axcolor = 'lightgoldenrodyellow'
ax.margins(x=0)

# Make a horizontal slider to control kappa.
axkappa_a = plt.axes([0.2, 0.1, 0.7, 0.03], facecolor=axcolor)
kappa_a_slider = Slider(
    ax=axkappa_a,
    label='$\kappa_A$',
    valmin=0,
    valmax=1,
    valinit=init_kappa_a,
)

# Make a horizontal slider to control kappa.
axkappa_b = plt.axes([0.2, 0.15, 0.7, 0.03], facecolor=axcolor)
kappa_b_slider = Slider(
    ax=axkappa_b,
    label='$\kappa_B$',
    valmin=0,
    valmax=1,
    valinit=init_kappa_b,
)

# Make a horizontal slider to control kappa_c
axkappa_c = plt.axes([0.2, 0.20, 0.7, 0.03], facecolor=axcolor)
kappa_c_slider = Slider(
    ax=axkappa_c,
    label="$\kappa_C$",
    valmin=0,
    valmax=1,
    valinit=init_kappa_c,
    orientation="horizontal"
)

# Make a hhorizontal oriented slider to control deltaL
axdeltaL = plt.axes([0.2, 0.25, 0.7, 0.03], facecolor=axcolor)
deltaL_slider = Slider(
    ax=axdeltaL,
    label=r"$\Delta L$ (% of $L_1^B$)",
    valmin=-0.002,
    valmax=0.002,
    valinit=init_deltaL,
    orientation="horizontal"
)

# Make a horizontal oriented slider to control deltaL
ax_a = plt.axes([0.2, 0.30, 0.7, 0.03], facecolor=axcolor)
a_slider = Slider(
    ax=ax_a,
    label=r"$\alpha$",
    valmin=0,
    valmax=500,
    valinit=init_a,
    orientation="horizontal"
)

# The function to be called anytime a slider's value changes
def update(val):  
    # Get the updated length
    new_L2B = L1B + deltaL_slider.val*L1B

    # Calculate the new profiles

    new_power_array_lossless = np.empty(wl_x.shape)
    for i, wl in enumerate(wl_x):

        # The other two variables are the complex field and the phase
        power_lossless = calculate_field(wl, 
                           n_r, n_w, 0,
                           kappa_a_slider.val, kappa_b_slider.val, kappa_c_slider.val,
                           L1A, L1B, L2A, new_L2B, L_w, 
                           field_option='R', 
                           x_option='wl', 
                          )
        new_power_array_lossless[i] = power_lossless
    R_lossless_line.set_data(wl_x*1e6,new_power_array_lossless)
    
    new_power_array = np.empty(wl_x.shape)
    for i, wl in enumerate(wl_x):
        # The other two variables are the complex field and the phase
        new_power = calculate_field(wl, 
            n_r, n_w, a_slider.val,
            kappa_a_slider.val, kappa_b_slider.val, kappa_c_slider.val, 
            L1A, L1B, L2A, new_L2B, L_w, 
            field_option='R',
            x_option='wl'
        )
        new_power_array[i] = new_power

    #Update the drawn line by changing the data
    R_line.set_data(wl_x*1e6,new_power_array)
    
    #Redraw
    fig.canvas.draw_idle()

# Run the update function everytime a slider is changed
kappa_a_slider.on_changed(update)
kappa_b_slider.on_changed(update)
kappa_c_slider.on_changed(update)
deltaL_slider.on_changed(update)
a_slider.on_changed(update)


# Create a `matplotlib.widgets.Button` to reset the sliders to initial values.
resetax = plt.axes([0.8, 0.025, 0.1, 0.04])
button = Button(resetax, 'Reset', color=axcolor, hovercolor='0.975')


def reset(event):
    kappa_a_slider.reset()
    kappa_b_slider.reset()
    kappa_c_slider.reset()
    deltaL_slider.reset()
    a_slider.reset()
button.on_clicked(reset)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0